# I. 영어 스팸 문자 데이터 탐색적 분석(EDA)

---
### 1) 데이터 로드 : dataframe (pandas module 사용)  
https://www.kaggle.com/uciml/sms-spam-collection-dataset  
train과 test가 별도로 나누어 지지 않았고, spam.csv 하나만 있다.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # 경고 메시지 안보이게 설정

import gc
gc.collect() # garbage collector : 메모리 관리

In [ ]:
import os

---
전역 변수 중 일부(디렉토리 이름과 파일 이름 등)는 대문자로  
나머지 변수는 소문자로

In [ ]:
# 파일 경로는 단순히 문자열 연결보다는 os.path.join()을 사용하는 것이 좋음 
DATA_DIR = 'data'
RAW_DATA_FILE = 'spam.csv'
RAW_DATA_PATH = os.path.join(DATA_DIR, RAW_DATA_FILE)

In [ ]:
import pandas as pd

In [ ]:
raw_df = pd.read_csv(RAW_DATA_PATH, encoding='latin-1')

In [ ]:
raw_df.shape

In [ ]:
raw_df.head()

In [ ]:
raw_df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)

In [ ]:
raw_df.head()

In [ ]:
raw_df.info()

In [ ]:
raw_df.describe()

---
### 2) 결측 데이터 처리

In [ ]:
print(raw_df.isnull().values.any())
print(raw_df.isnull().sum())
raw_df.isnull().sum()[raw_df.isnull().sum().values > 0]

# 결측 데이터 없음.

---
### 3) 중복 데이터 제거

In [ ]:
duplicated_row = raw_df[raw_df.duplicated('v2')]
print(duplicated_row[:5])
# duplicatedRow
len(duplicated_row)

In [ ]:
raw_df.groupby('v1').describe().T

In [ ]:
dedupe_raw_df = raw_df.drop_duplicates('v2', keep='first')

In [ ]:
duplicated_row_dedupe = dedupe_raw_df[dedupe_raw_df.duplicated('v2')]
print(duplicated_row_dedupe[:5])
# duplicatedRow
len(duplicated_row_dedupe)

In [ ]:
dedupe_raw_df.shape

---
### 4) EDA (Exploratory Data Analysis; 탐색적 데이터 분석)

In [ ]:
from collections import Counter #데이터 전처리

Counter(dedupe_raw_df['v1'])

In [ ]:
print(653/4516)
print(653/(653 + 4516))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# spam과 ham의 수를 막대 그래프로 비교

fig, axe = plt.subplots(ncols=1)
fig.set_size_inches(6, 3)
sns.countplot(dedupe_raw_df['v1'])

---
(가) 문장의 길이(글자수) 분포 조사

In [ ]:
# 메시지의 길이(글자 수)가 어떤 값들을 갖는지 분석
raw_length = dedupe_raw_df['v2'].apply(len)

raw_length.head()

In [ ]:
dedupe_raw_df['v2'][0]

In [ ]:
raw_length.describe()

In [ ]:
import numpy as np

print('문자 길이 최대 값: {}'.format(np.max(raw_length)))
print('문자 길이 최소 값: {}'.format(np.min(raw_length)))
print('문자 길이 평균 값: {:.2f}'.format(np.mean(raw_length)))
print('문자 길이 표준편차: {:.2f}'.format(np.std(raw_length)))
print('문자 길이 중간 값: {}'.format(np.median(raw_length)))
# 사분위의 대한 경우는 0~100 스케일로 되어있음
print('문자 길이 제 1 사분위: {}'.format(np.percentile(raw_length, 25))) # nanpercentile()을 쓰면 NaN 값이 있을 때도 에러 안남.
print('문자 길이 제 3 사분위: {}'.format(np.percentile(raw_length, 75))) # nanpercentile()을 쓰면 NaN 값이 있을 때도 에러 안남.

In [ ]:
# 그래프에 대한 이미지 사이즈 선언
# figsize: (가로, 세로) 형태의 튜플로 입력
plt.figure(figsize=(12, 5))
# 히스토그램 선언
# bins: 히스토그램 값들에 대한 버켓 범위
# range: x축 값의 범위
# alpha: 그래프 색상 투명도
# color: 그래프 색상
# label: 그래프에 대한 라벨
plt.hist(raw_length, bins=200, facecolor='r', label='# of char')
plt.yscale('log', nonpositive='clip')
plt.legend()
# 그래프 제목
plt.title('Log-Histogram of length of text')
# 그래프 x 축 라벨
plt.xlabel('Length of text')
# 그래프 y 축 라벨
plt.ylabel('Number of text')

In [ ]:
plt.figure(figsize=(12, 5))
# 박스플롯 생성
# 첫번째 파라메터: 여러 분포에 대한 데이터 리스트를 입력
# labels: 입력한 데이터에 대한 라벨
# showmeans: 평균값을 마크함

plt.boxplot(raw_length,
             labels=['counts'],
             showmeans=True)

---
#### (나) 단어 수 분포 조사  

In [ ]:
# 영문 데이터이므로 비칸을 기준으로 토크나이즈하고 카운팅
raw_word_counts = dedupe_raw_df['v2'].apply(lambda x:len(x.split(' ')))

In [ ]:
raw_word_counts.head()

In [ ]:
raw_word_counts.describe()

In [ ]:
print('문자 단어 개수 최대 값: {}'.format(np.max(raw_word_counts)))
print('문자 단어 개수 최소 값: {}'.format(np.min(raw_word_counts)))
print('문자 단어 개수 평균 값: {:.2f}'.format(np.mean(raw_word_counts)))
print('문자 단어 개수 표준편차: {:.2f}'.format(np.std(raw_word_counts)))
print('문자 단어 개수 중간 값: {}'.format(np.median(raw_word_counts)))
# 사분위의 대한 경우는 0~100 스케일로 되어있음
print('문자 단어 개수 제 1 사분위: {}'.format(np.percentile(raw_word_counts, 25)))
print('문자 단어 개수 제 3 사분위: {}'.format(np.percentile(raw_word_counts, 75)))

In [ ]:
plt.figure(figsize=(12, 5))
plt.hist(raw_word_counts, range=(1, 172), bins=171, facecolor='r', label='# of words')
plt.title('Log-Histogram of word count in text', fontsize=15)
plt.yscale('log', nonpositive='clip')
plt.legend()
plt.xlabel('Number of words', fontsize=15)
plt.ylabel('Number of text', fontsize=15)

In [ ]:
qmarks = np.mean(dedupe_raw_df['v2'].apply(lambda x: '?' in x)) # 물음표가 구두점으로 쓰임
fullstop = np.mean(dedupe_raw_df['v2'].apply(lambda x: '.' in x)) # 마침표
capital_first = np.mean(dedupe_raw_df['v2'].apply(lambda x: x[0].isupper())) #  첫번째 대문자
capitals = np.mean(dedupe_raw_df['v2'].apply(lambda x: max([y.isupper() for y in x]))) # 대문자가 몇개
numbers = np.mean(dedupe_raw_df['v2'].apply(lambda x: max([y.isdigit() for y in x]))) # 숫자가 몇개
                  
print('물음표가 있는 문장: {:.2f}%'.format(qmarks * 100))
print('마침표가 있는 문장: {:.2f}%'.format(fullstop * 100))
print('첫 글자가 대문자 인 문장: {:.2f}%'.format(capital_first * 100))
print('대문자가 있는 문장: {:.2f}%'.format(capitals * 100))
print('숫자가 있는 문장: {:.2f}%'.format(numbers * 100))

---
### 5) 워드 클라우드 표시

In [ ]:
# Get all the ham and spam
ham_msg = dedupe_raw_df[dedupe_raw_df.v1 == 'ham']
spam_msg = dedupe_raw_df[dedupe_raw_df.v1 == 'spam']

In [ ]:
# 전체 문장을 모두 이어 붙임.

# print(pd.__version__) 버전이 0.24.1 이상이면 아래와 같이 사용 가능
# spam_msg_text = " ".join(spam_msg.v2.to_numpy().tolist())

# 버전이 0.24.1 보다 낮으면 아래와 같이 사용

# Create numpy list to visualize using wordcloud
ham_msg_text = " ".join(np.array(ham_msg.v2).tolist())
spam_msg_text = " ".join(np.array(spam_msg.v2).tolist())

In [ ]:
# !pip install wordcloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS

In [ ]:
# wordcloud of ham messages
ham_msg_cloud = WordCloud(width =520, height =260, stopwords=STOPWORDS, max_font_size=50, background_color ="black", colormap='Blues').generate(ham_msg_text)
plt.figure(figsize=(16,10))
plt.imshow(ham_msg_cloud, interpolation='bilinear')
plt.axis('off') # turn off axis
plt.show()

In [ ]:
# wordcloud of spam messages
spam_msg_cloud = WordCloud(width =520, height =260, stopwords=STOPWORDS, max_font_size=50, background_color ="black", colormap='Blues').generate(spam_msg_text)
plt.figure(figsize=(16,10))
plt.imshow(spam_msg_cloud, interpolation='bilinear')
plt.axis('off') # turn off axis
plt.show()